In [77]:
import json
import pandas as pd
import geopandas as gpd
import numpy as np
from uuid import uuid4
import datetime as dt
import subprocess

In [99]:
file = 'data/F2-2022_07_06-1407.BIN'
outfile = open(file.split('.')[0]+'.json', 'w')
subprocess.call(['python', 'mavlogdump.py', '--format=json',file], stdout=outfile, shell=True)

0

In [101]:
with open(outfile.name, 'r') as f:
    raw_str = f.readlines()
pos_str = [json.loads(line) for line in raw_str if 'POS' in line]
pos_df = pd.json_normalize(pos_str)
pos_df

,meta.type,meta.timestamp,data.Type,data.Length,data.Name,data.Format,data.Columns,data.TimeUS,data.Lat,data.Lng,data.Alt,data.RelHomeAlt,data.RelOriginAlt,data.Value
0,FMT,1.657118e+09,173.0,31.0,POS,QLLfff,"TimeUS,Lat,Lng,Alt,RelHomeAlt,RelOriginAlt",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,POS,1.657116e+09,NaN,NaN,NaN,NaN,NaN,7.415264e+07,52.811266,-4.123635,8.65,-0.010395,-0.010395,NaN
2,PARM,1.657116e+09,NaN,NaN,GPS1_POS_X,NaN,NaN,7.415851e+07,NaN,NaN,NaN,NaN,NaN,0.0
3,PARM,1.657116e+09,NaN,NaN,GPS1_POS_Y,NaN,NaN,7.415854e+07,NaN,NaN,NaN,NaN,NaN,0.0
4,PARM,1.657116e+09,NaN,NaN,GPS1_POS_Z,NaN,NaN,7.415857e+07,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65438,POS,1.657119e+09,NaN,NaN,NaN,NaN,NaN,2.712414e+09,52.811296,-4.123519,11.99,-0.135292,-0.135292,NaN
65439,POS,1.657119e+09,NaN,NaN,NaN,NaN,NaN,2.712454e+09,52.811296,-4.123519,11.99,-0.137154,-0.137154,NaN
65440,POS,1.657119e+09,NaN,NaN,NaN,NaN,NaN,2.712493e+09,52.811296,-4.123519,11.99,-0.138925,-0.138925,NaN
65441,POS,1.657119e+09,NaN,NaN,NaN,NaN,NaN,2.712535e+09,52.811296,-4.123519,11.98,-0.140652,-0.140652,NaN


In [102]:
pos_df = pos_df[pos_df['meta.type'] == 'POS'].dropna(axis=1).drop_duplicates(
    subset=['data.Lat', 'data.Lng', 'data.Alt']).drop(
    labels=['meta.type', 'data.TimeUS', 'data.RelHomeAlt', 'data.RelOriginAlt'], axis=1)
pos_gdf = gpd.GeoDataFrame(pos_df,
                           geometry=gpd.points_from_xy(pos_df['data.Lng'], pos_df['data.Lat'], crs='epsg:4326'))



In [104]:
pos_df = pos_gdf[['meta.timestamp', 'data.Lng', 'data.Lat', 'data.Alt']]
flat_4d_coords = list(np.array(pos_gdf[['meta.timestamp', 'data.Lng', 'data.Lat', 'data.Alt']].values).flatten())
flat_3d_coords = list(np.array(pos_gdf[['data.Lng', 'data.Lat', 'data.Alt']].values).flatten())
max_time = dt.datetime.fromtimestamp(pos_df['meta.timestamp'].max())
min_time = dt.datetime.fromtimestamp(pos_df['meta.timestamp'].min())
uid = str(uuid4())
pos_df

,meta.timestamp,data.Lng,data.Lat,data.Alt
1,1.657116e+09,-4.123635,52.811266,8.65
24,1.657116e+09,-4.123635,52.811266,8.66
29,1.657116e+09,-4.123635,52.811266,8.65
46,1.657116e+09,-4.123635,52.811266,8.66
48,1.657116e+09,-4.123635,52.811266,8.66
...,...,...,...,...
65429,1.657119e+09,-4.123519,52.811296,12.00
65434,1.657119e+09,-4.123519,52.811296,11.99
65437,1.657119e+09,-4.123519,52.811296,11.99
65438,1.657119e+09,-4.123519,52.811296,11.99


In [105]:
czml_id_packet = {
    'id': "document",
    'name': "CZML Model",
    'version': "1.0",
}

In [106]:
czml_model_packet = {
    'id': uid + '.log',
    'availability': min_time.replace(microsecond=0).isoformat() + 'Z/' + max_time.replace(
        microsecond=0).isoformat() + 'Z',
    'position': {"epoch": "1970-01-01T00:00:00Z",
                 "cartographicDegrees": flat_4d_coords},
    'model': {
        "gltf": "https://raw.githubusercontent.com/bobbyhiom/cyberpunkjam/master/CyberPunkScene/models/drone.gltf",
        "scale": 1, "minimumPixelSize": 32}
}

In [107]:
czml_traj_packet = {
    'id': uid + '.log',
    'availability': min_time.replace(microsecond=0).isoformat() + 'Z/' + max_time.replace(
        microsecond=0).isoformat() + 'Z',
    'polyline': {"positions": {
        "cartographicDegrees": flat_3d_coords
    },
        "material": {"solidColor": {"color": {"rgba": [200, 255, 200, 255]}}}}
}

In [71]:
with open('spotter.czml', 'w') as f:
    # f.write(str(czml_id_packet))
    # f.write(',')
    # f.write(str(czml_model_packet))
    # f.write(',')
    f.write(str(czml_traj_packet))

In [67]:
from czml3.widget import CZMLWidget
from czml3.examples import simple